<a href="https://colab.research.google.com/github/Annisaard/Machine-Learning-homework/blob/main/Week9/Week_9__chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.d2l.ai/chapter_linear-networks/linear-regression-concise.html

# Concise Implementation of Linear Regression

In [1]:
!pip install -U mxnet-cu101==1.7.0

     |███████████████████████████████▌| 834.1 MB 1.1 MB/s eta 0:00:11tcmalloc: large alloc 1147494400 bytes == 0x563d5b672000 @  0x7f57adee2615 0x563d2130a4cc 0x563d213ea47a 0x563d2130d2ed 0x563d213fee1d 0x563d21380e99 0x563d2137b9ee 0x563d2130ebda 0x563d21380d00 0x563d2137b9ee 0x563d2130ebda 0x563d2137d737 0x563d213ffc66 0x563d2137cdaf 0x563d213ffc66 0x563d2137cdaf 0x563d213ffc66 0x563d2137cdaf 0x563d2130f039 0x563d21352409 0x563d2130dc52 0x563d21380c25 0x563d2137b9ee 0x563d2130ebda 0x563d2137d737 0x563d2137b9ee 0x563d2130ebda 0x563d2137c915 0x563d2130eafa 0x563d2137cc0d 0x563d2137b9ee
     |████████████████████████████████| 846.0 MB 20 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
!pip install d2l==0.17.0

     |████████████████████████████████| 83 kB 1.3 MB/s 


In [3]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)


# Reading Dataset

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)


In [5]:
next(iter(data_iter))


[array([[-1.2501998 ,  0.6437783 ],
        [-0.03716067, -0.13531584],
        [ 1.4418106 , -1.4980611 ],
        [ 0.2654874 ,  0.74110895],
        [-0.55512375,  0.16916618],
        [ 0.46009848, -0.20287366],
        [ 0.03476851,  0.3742453 ],
        [-0.51905143, -2.067554  ],
        [-0.06495   , -0.20424576],
        [ 0.51737875,  1.6076493 ]]), array([[-0.46014845],
        [ 4.588353  ],
        [12.180052  ],
        [ 2.2031896 ],
        [ 2.5186574 ],
        [ 5.805376  ],
        [ 3.0018418 ],
        [10.181485  ],
        [ 4.778515  ],
        [-0.22868225]])]

# Defining the Model

In [6]:
# `nn` is an abbreviation for neural networks
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))


# initializing Model Parameters

In [7]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))


# Defining the Loss Function

In [8]:
loss = gluon.loss.L2Loss()


# Defining the Optimization Algorithm

In [9]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})


# Training

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')


epoch 1, loss 0.025074
epoch 2, loss 0.000086
epoch 3, loss 0.000051


In [11]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')


error in estimating w: [0.00051701 0.00014973]
error in estimating b: [0.00082111]
